<a href="https://colab.research.google.com/github/nguyentantan141/MACHINE-LEARNING/blob/Machine-learning/k_fold_CV_Logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import numpy as np
import os
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ShuffleSplit

In [59]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Đọc dữ liệu

In [60]:
def readData(pathfolder: str, filename: str):
  data=np.loadtxt(os.path.join(pathfolder, filename),delimiter=',')
  X, y = data[:,:-1], data[:, -1]
  return X, y

In [61]:
def scaling_minmax_norm(X):
  scaler =MinMaxScaler()
  #Phải thực hiện thao tác fit(data) trước khi điều chỉnh dữ liệu
  scaler.fit(X)
  #Thực hiện điều chỉnh dữ liệu
  X = scaler.transform(X)
  m=X.shape[0]
  X_scl=np.hstack((np.ones((m,1)),X))
  return X_scl
  

In [62]:
def split_data(X_scl,y):
  X_train, X_test, y_train, y_test = train_test_split(X_scl, y,test_size=0.30, random_state=15)
  return X_train, X_test, y_train, y_test

# Huấn luyện mô hình

In [83]:
def train_model(X_train,X_test,y_train,y_test):
  #Huấn luyện mô hình
  #Số lượng k-fold được xác định tùy thuộc vào số lượng y_train
  unique, counts = np.unique(y_train, return_counts=True)
  result = dict(zip(unique, counts))
  print(result)
  print('Huấn luyện mô hình Logistic  với  10_fold_cv: ')
  model = LogisticRegression(solver='liblinear',max_iter=1500,
                             multi_class='auto') # Thay thế solver với các thuật toán tương tự: lbfgs, sag, saga,.. 
  cv=ShuffleSplit(n_splits=10, test_size=0.2, random_state=15)
  scores=cross_val_score(model, X_train, y_train,cv=cv,scoring='accuracy') # thay thế các chỉ số tương tự: balanced_accuracy, top_k_accuracy.
  y_hat=cross_val_predict(model, X_train, y_train,cv=10)
  #Tìm bộ tham số tối ưu cho mô hình bằng GridsearchCV
  #Đặc tả 10-fold cv với k = 10
  parameters = {'C':[1, 10, 20, 50]}
  cv = GridSearchCV(model, parameters, cv=10)

  #Huấn luyện mô hình và lấy kết quả tối ưu
  cv.fit(X_train, y_train)
  #Thông báo kết quả tối ưu
  print('Kết quả huấn luyên 10-fold cv')
  print('\t', cv.best_params_)

  #Tạo mô hình LogisticRegression với best params
  model.set_params(**cv.best_params_)
  model.fit(X_train, y_train)
  y_hat = cv.predict(X_test)

  #Bước 9 đánh giá hiệu năng mô hình
  print('Hiệu năng mô hình acc: ', accuracy_score(y_hat, y_test))

  return scores, y_hat
  


# Sử dụng hàm cross_val_score chỉ trả về độ chính xác của mô hình, cross_val_predict chỉ trả về kết quả dự đoán của tập X_test truyền vào. Hai hàm này không trả được mô hình chúng ta muốn tìm

In [82]:
def main(pathfolder: str, filename: str):
  X,y=readData(pathfolder, filename)
  X_scl=scaling_minmax_norm(X)
  X_train, X_test, y_train, y_test=split_data(X_scl, y)
  scores,y_hat=train_model(X_train, X_test, y_train, y_test)
  print("scores:\n",scores)
  print("y_hat:\n",y_hat)
  
if __name__ == '__main__':
  main('/content/drive/MyDrive/Data/data/','ex2data2.txt')

{0.0: 41, 1.0: 41}
Huấn luyện mô hình Logistic  với  10_fold_cv: 
Kết quả huấn luyên 10-fold cv
	 {'C': 1}
Hiệu năng mô hình acc:  0.5
scores:
 [0.41176471 0.29411765 0.47058824 0.52941176 0.52941176 0.52941176
 0.47058824 0.41176471 0.41176471 0.35294118]
y_hat:
 [0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0.]
